In [1]:
%load_ext autoreload

In [7]:
%autoreload 2

from pathlib import Path

from src.config_loader import config
from src.utils import anthropic_generate, upload_to_gcs, load_json, extract_json_from_llm_response, save_json, list_story_folders
from src.chat import get_challenge_generation_prompt, create_html_challenges, get_html_challenge_inputs

In [8]:
all_stories = list_story_folders()
all_stories

['story_birthday_party_planning_mishap',
 'story_camping_trip_gone_awry',
 'story_community_park',
 'story_dining_dilemma_at_local_restaurant',
 'story_fishing_trip_gone_awry',
 'story_job_interview_gone_wrong',
 'story_lost_in_stockholm',
 'story_roblox_bot_trouble',
 'story_swedish_adventure_in_winter_wilderness',
 'story_underwater_adventure_gone_wrong']

In [20]:
STORY_NAME = "story_lost_in_stockholm"
notebook_dir = Path().absolute()  # This gives src/notebooks
phrase_dir = notebook_dir.parent / "data" / "phrases" #where we store text files of phrases
story_dir = notebook_dir.parent / "outputs" / "stories" / STORY_NAME 
chat_dir = story_dir / config.TARGET_LANGUAGE_NAME
story_dialgoue_file = story_dir / f"{STORY_NAME}.json"
print(config.TARGET_LANGUAGE_NAME)

Swedish


## Generate scenarios

In [21]:
story_dialogue = load_json(story_dialgoue_file)

JSONDecodeError: Expecting value: line 56 column 7 (char 1933)

In [17]:
prompt = get_challenge_generation_prompt(story_dialogue)
scenario_dicts = extract_json_from_llm_response(anthropic_generate(prompt, max_tokens=5000))

save_json(data=scenario_dicts, file_path=f"{story_dir}/scenarios.json")

Function that called this one: <module>. Sleeping for 20 seconds


## If scenarios already exist start here

In [18]:
scenario_dicts = load_json(file_path=f"{story_dir}/scenarios.json")
challenges = get_html_challenge_inputs(scenario_dicts) # this is the first time config.TARGET_LANGUAGE_NAME is used
chat_webpage_file = create_html_challenges(challenges, output_dir=chat_dir,
story_name=STORY_NAME)


HTML challenges created at: y:\Python Scripts\audio-language-trainer\outputs\stories\story_community_park\Swedish\challenges.html


In [19]:
upload_to_gcs(file_path=chat_webpage_file, bucket_prefix=f"{config.TARGET_LANGUAGE_NAME}/{STORY_NAME}")

'https://storage.googleapis.com/audio-language-trainer-stories/swedish/story_community_park/challenges.html'